In [2]:
import os 
import pandas as pd

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [7]:
df.drop( columns = ['rating' , 'timestamp'] , inplace = True)

df.head()

,userId,movieId
0,1,17
1,1,25
2,1,29
3,1,30
4,1,32


In [8]:
df.describe()

,userId,movieId
count,3.200020e+07,3.200020e+07
mean,1.002785e+05,2.931861e+04
std,5.794905e+04,5.095816e+04
min,1.000000e+00,1.000000e+00
25%,5.005300e+04,1.233000e+03
50%,1.002970e+05,3.452000e+03
75%,1.504510e+05,4.419900e+04
max,2.009480e+05,2.927570e+05


In [16]:
%pip install scikit-learn


  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)



  Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.2 MB)


  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)



Note: you may need to restart the kernel to use updated packages.


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 32000204 entries, 0 to 32000203

Data columns (total 2 columns):

 #   Column   Dtype

---  ------   -----

 0   userId   int64

 1   movieId  int64

dtypes: int64(2)

memory usage: 488.3 MB


In [19]:
#now splliting the data into training and testing split: 
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.to_csv('train_data.txt' , index = False)
test_df.to_csv('test_data.txt', index=False)


train_df.info()
test_df.info()

In [1]:
import numpy as np
import math

def topK_scores(test, predict, topk, user_count, item_count):
    PrecisionSum = np.zeros(topk+1)
    RecallSum = np.zeros(topk+1)
    F1Sum = np.zeros(topk+1)
    NDCGSum = np.zeros(topk+1)
    OneCallSum = np.zeros(topk+1)
    MRRSum = 0
    MAPSum = 0
    total_test_data_count = 0
    
    # Precompute DCGbest for efficiency
    DCGbest = np.zeros(topk+1)
    for k in range(1, topk+1):
        DCGbest[k] = DCGbest[k - 1] + 1.0 / math.log2(k + 1)
    
    # Loop over each user
    for i in range(user_count):
        user_test = test[i * item_count:(i + 1) * item_count]
        user_predict = predict[i * item_count:(i + 1) * item_count]

        # Get test data size for this user
        test_data_size = np.sum(user_test)
        if test_data_size == 0:
            continue
        total_test_data_count += 1

        # Get top-k item indices for this user
        top_k_indices = np.argsort(user_predict)[-topk:][::-1]  # Sort in descending order
        
        hit_sum = 0
        DCG = np.zeros(topk + 1)
        for k in range(1, topk + 1):
            item_id = top_k_indices[k - 1]
            if user_test[item_id] == 1:
                hit_sum += 1
                DCG[k] = DCG[k - 1] + 1 / math.log2(k + 1)
            else:
                DCG[k] = DCG[k - 1]
            
            prec = hit_sum / k
            rec = hit_sum / test_data_size
            f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0

            PrecisionSum[k] += prec
            RecallSum[k] += rec
            F1Sum[k] += f1
            NDCGSum[k] += DCG[k] / DCGbest[k]
            OneCallSum[k] += 1 if hit_sum > 0 else 0

        # Compute MRR
        for rank, idx in enumerate(top_k_indices, start=1):
            if user_test[idx] == 1:
                MRRSum += 1 / rank
                break

        # Compute MAP
        AP = 0
        hit_count = 0
        for rank, idx in enumerate(top_k_indices, start=1):
            if user_test[idx] == 1:
                hit_count += 1
                AP += hit_count / rank
        MAPSum += AP / test_data_size
    
    # Normalize metrics
    total_test_data_count = max(1, total_test_data_count)  # Avoid division by 0
    print('MAP:', MAPSum / total_test_data_count)
    print('MRR:', MRRSum / total_test_data_count)
    print('Prec@5:', PrecisionSum[5] / total_test_data_count)
    print('Rec@5:', RecallSum[5] / total_test_data_count)
    print('F1@5:', F1Sum[5] / total_test_data_count)
    print('NDCG@5:', NDCGSum[5] / total_test_data_count)
    print('1-call@5:', OneCallSum[5] / total_test_data_count)


In [4]:
import pandas as pd
import os
from collections import defaultdict

# Load the train and test data
train_df = pd.read_csv("/kaggle/input/new-dataset/train_data.txt", delimiter=",")
test_df = pd.read_csv("/kaggle/input/new-dataset/test_data.txt", delimiter=",")

# Convert the dataframes to lists of tuples
train_user_item_pairs = list(train_df.itertuples(index=False, name=None))  # [(userid, movieid), ...]
test_user_item_pairs = list(test_df.itertuples(index=False, name=None))  # [(userid, movieid), ...]

# Define the filtering function
def filter_data(user_item_pairs, min_interactions=10):
    # Step 1: Count interactions
    user_counts = defaultdict(set)
    item_counts = defaultdict(set)
    
    for user, item in user_item_pairs:
        user_counts[user].add(item)
        item_counts[item].add(user)

    # Step 2: Filter out users with less than `min_interactions` items
    filtered_users = {user: items for user, items in user_counts.items() if len(items) >= min_interactions}
    
    # Step 3: Filter out items with less than `min_interactions` users
    filtered_items = {item: users for item, users in item_counts.items() if len(users) >= min_interactions}

    # Step 4: Remove any remaining users/items that no longer meet the conditions
    while True:
        # Remove items from users that don't exist in filtered items
        new_filtered_users = {user: {item for item in items if item in filtered_items} for user, items in filtered_users.items()}
        # Remove users that now have fewer than `min_interactions` items
        new_filtered_users = {user: items for user, items in new_filtered_users.items() if len(items) >= min_interactions}
        
        # Remove users from items that don't exist in filtered users
        new_filtered_items = {item: {user for user in users if user in new_filtered_users} for item, users in filtered_items.items()}
        # Remove items that now have fewer than `min_interactions` users
        new_filtered_items = {item: users for item, users in new_filtered_items.items() if len(users) >= min_interactions}

        # Check if the filtering stabilized
        if new_filtered_users == filtered_users and new_filtered_items == filtered_items:
            break
        
        filtered_users, filtered_items = new_filtered_users, new_filtered_items

    # Convert filtered data back to a list of duplets
    filtered_user_item_pairs = [(user, item) for user, items in filtered_users.items() for item in items]
    
    return filtered_user_item_pairs

# Apply filtering to train and test data
filtered_train_pairs = filter_data(train_user_item_pairs)
filtered_test_pairs = filter_data(test_user_item_pairs)

# Convert the filtered pairs back to DataFrames
filtered_train_df = pd.DataFrame(filtered_train_pairs, columns=['userid', 'movieid'])
filtered_test_df = pd.DataFrame(filtered_test_pairs, columns=['userid', 'movieid'])

# Overwrite the filtered DataFrames to their respective text files
filtered_train_df.to_csv("/kaggle/working/train_data_filtered.txt", index=False, header=False)
filtered_test_df.to_csv("/kaggle/working/test_data_filtered.txt", index=False, header=False)

print("Filtered train and test data saved.")

Filtered train and test data saved.


In [5]:
remaining_users = filtered_train_df['userid'].nunique()  # Count unique users
remaining_items = filtered_train_df['movieid'].nunique()  # Count unique items

print(f"Remaining users: {remaining_users}")
print(f"Remaining items: {remaining_items}")

Remaining users: 100646
Remaining items: 21508


In [6]:
filtered_train_df = filtered_train_df.sample(frac=0.001, random_state=42)
filtered_test_df=  filtered_test_df.sample(frac=0.001, random_state=42)

In [7]:
import random
from collections import defaultdict
# import cupy as cp  # Import CuPy for GPU support
from sklearn.metrics import roc_auc_score

class BPR:
    def __init__(self, filtered_user_count, unique_item_count, latent_factors, lr, reg, filtered_train_df, filtered_test_df):
        self.user_count = filtered_user_count
        self.item_count = unique_item_count  # Unique item count
        self.latent_factors = latent_factors
        self.lr = lr
        self.reg = reg
        self.filtered_train_df = filtered_train_df  # Using filtered train DataFrame
        self.filtered_test_df = filtered_test_df  # Using filtered test DataFrame

        # Initialize latent factors for users and items
        self.U = cp.random.rand(self.user_count, self.latent_factors).astype(cp.float32) * 0.01
        self.V = cp.random.rand(self.item_count, self.latent_factors).astype(cp.float32) * 0.01
        self.biasV = cp.random.rand(self.item_count).astype(cp.float32) * 0.01
        
        # Initialize test data using a dictionary for sparse representation
        self.test_data = defaultdict(dict)  # Maps user index to item index with values
        self.predict_ = cp.zeros(self.user_count * self.item_count, dtype=cp.float32)

        # Initialize user and item mappings
        self.user_mapping = {}
        self.item_mapping = {}
        self.reverse_user_mapping = {}
        self.reverse_item_mapping = {}

        # Create user and item mappings from filtered DataFrames
        self.create_mappings()

    def create_mappings(self):
        user_id_set = self.filtered_train_df['userid'].unique()  # Extract unique user IDs
        item_id_set = self.filtered_train_df['movieid'].unique()  # Extract unique item IDs

        # Create a mapping for user IDs
        for index, user_id in enumerate(user_id_set):
            self.user_mapping[user_id] = index
            self.reverse_user_mapping[index] = user_id

        # Create a mapping for item IDs
        for index, item_id in enumerate(item_id_set):
            self.item_mapping[item_id] = index
            self.reverse_item_mapping[index] = item_id

    def load_test_data(self):
        for _, row in self.filtered_test_df.iterrows():
            user = int(row['userid'])
            item = int(row['movieid'])

            if user in self.user_mapping:  # Only process if user is in mapping
                # Get the mapped index
                mapped_user_index = self.user_mapping[user]
                
                if item in self.item_mapping:  # Check if the item is mapped
                    mapped_item_index = self.item_mapping[item]
                    # Use dictionary for test data
                    self.test_data[mapped_user_index][mapped_item_index] = 1.0  # Mark interaction

    def train(self, user_ratings_train, bootstrap=True):
        
            for _ in range(self.user_count):  # Number of iterations
                try: 
                    # Bootstrap sample a user
                    u_original = random.choice(list(user_ratings_train.keys())) if bootstrap else random.randint(1, self.user_count)
                    if u_original not in user_ratings_train:
                        continue

                    # Map to the new user index
                    u = self.user_mapping[u_original]

                    # Sample a positive item (i) from user's interactions
                    i = random.sample(user_ratings_train[u_original], 1)[0]  # Convert to list for sampling

                    # Sample a negative item (j) not interacted by user
                    j = random.randint(1, self.item_count)
                    while j in user_ratings_train[u_original]:
                        j = random.randint(1, self.item_count)

                    # Decrement to make zero-indexed
                    i = self.item_mapping[i]  # Map to new item index
                    j = self.item_mapping[j]  # Map to new item index

                    # Compute predictions for positive (i) and negative (j) items
                    r_ui = cp.dot(self.U[u], self.V[i]) + self.biasV[i]
                    r_uj = cp.dot(self.U[u], self.V[j]) + self.biasV[j]
                    r_uij = r_ui - r_uj

                    # BPR optimization: Compute the gradient and update using log-sigmoid
                    loss_func = -1.0 / (1 + cp.exp(r_uij))  # Logistic function for BPR loss

                    # Update latent factors for user and items
                    self.U[u] += self.lr * (loss_func * (self.V[i] - self.V[j]) - self.reg * self.U[u])
                    self.V[i] += self.lr * (loss_func * self.U[u] - self.reg * self.V[i])
                    self.V[j] += self.lr * (-loss_func * self.U[u] - self.reg * self.V[j])

                    # Update biases for items
                    self.biasV[i] += self.lr * (loss_func - self.reg * self.biasV[i])
                    self.biasV[j] += self.lr * (-loss_func - self.reg * self.biasV[j])
                except:
                    continue

    def predict(self):
        # Prediction matrix for user-item interactions
        predict_matrix = cp.dot(self.U, self.V.T) + self.biasV
        return predict_matrix

    def evaluate(self):
        # Evaluate using AUC score
        predictions = self.predict()
        # Flatten the test data using a comprehension since it's a dict
        test_flat = cp.array([self.test_data[u].get(i, 0) for u in range(self.user_count) for i in range(self.item_count)], dtype=cp.float32)
        auc_score = roc_auc_score(test_flat.get(), predictions.get())  # Convert CuPy arrays to NumPy arrays
        print('AUC:', auc_score)

    def main(self):
        user_ratings_train = defaultdict(set)
        
        # Prepare user ratings from filtered_train_df
        for _, row in self.filtered_train_df.iterrows():
            user = int(row['userid'])
            item = int(row['movieid'])
            user_ratings_train[user].add(item)

        self.load_test_data()

        # Train model
        for _ in range(1000):  # Number of training steps
            self.train(user_ratings_train)
        
        # Evaluate model
        self.evaluate()

# Running the BPR model
if __name__ == '__main__':
    filtered_train_df = filtered_train_df.sample(frac=0.001, random_state=42)
    filtered_test_df=  filtered_test_df.sample(frac=0.001, random_state=42)
    filtered_user_count = 10064  # Number of unique users after filtering
    latent_factors = 10  # Example value
    lr = 0.01  # Learning rate
    reg = 0.01  # Regularization term

    # Initialize BPR model with filtered DataFrames
    bpr = BPR(filtered_user_count, 2158, latent_factors, lr, reg, filtered_train_df, filtered_test_df)  # Using the correct unique item count
    bpr.main()


NameError: name 'cp' is not defined